In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
gtes = "gtes04ene.xls"
goi = "goi04ene.xls"
gom = "gom04ene.xls"
gef = "gef04ene.xls"
cuentas=[7,10,11,12,16,17]
bdgtes = pd.read_excel('../data/'+gtes)
bdgoi = pd.read_excel('../data/'+goi)
bdgom = pd.read_excel('../data/'+gom)
bdgef = pd.read_excel('../data/'+gef)

bdBruto= pd.concat([bdgtes,bdgoi,bdgom,bdgef],ignore_index=True)
#bd.describe()


In [2]:
bdBruto.columns

Index(['cod_unidad', 'glosa_reng', 'cve_debe_haber', 'monto_mo',
       'factor_conv_mo_mn', 'nro_centro', 'cod_moneda', 'cod_movimiento',
       'fecha_dia', 'nom_movimiento', 'nom_moneda', 'nom_persona',
       'exponente_escala', 'monto_mn', 'compute_0015', 'compute_0016',
       'nro_afectable', 'glosa_comprob', 'cve_tipo_comprob', 'nro_comprob',
       'nro_aprobacion', 'nro_reng', 'nom_centro', 'fecha_hora', 'cod_mayor',
       'nom_mayor', 'cod_tipo_cuenta', 'desc_tipo_cuenta', 'cod_persona',
       'cod_moneda.1'],
      dtype='object')

In [3]:
bdReducida=bdBruto.loc[:,('cve_tipo_comprob','glosa_reng','cve_debe_haber','monto_mo',
           'cod_moneda', 'cod_movimiento', 'nom_movimiento',
           'monto_mn','glosa_comprob','nro_comprob','cod_mayor')]
#bdReducida

In [4]:
dict = {'cve_tipo_comprob':'tipo','glosa_reng':'glosaRenglon','cve_debe_haber':'dh','monto_mo':'mo',
           'cod_moneda':'moneda', 'cod_movimiento':'codMov', 'nom_movimiento':'nomMov',
           'monto_mn':'mn','glosa_comprob':'glosa','nro_comprob':'comprobante','cod_mayor':'mayor'}
bdReducida.rename(columns=dict,
          inplace=True)
bdReducida.shape

(4815, 11)

In [5]:
#bd['mayor'].unique()

#bdFiltro1=bdReducida[bdReducida['mayor'].isin([7,10,11,16,17])]
bdFiltro1=bdReducida[bdReducida['mayor'].isin(cuentas)]

In [6]:
#bdFiltro1.loc[:,("comprobante","mayor")]
bdFiltro1["comprobante"].unique()
#bdFiltro1.sort_values(by="comprobante", ascending=True)

array([1988999, 1989052, 1989068, 1023079, 1023080, 1023086, 1023090,
         28350, 1023082,   21762, 1747929,   28354, 1747939, 1023099,
       1023081, 1747823, 1023083, 1747923, 1747924, 1747926, 1023106,
       1023110, 1747931, 1747933, 1747935, 1747937, 1023104, 1023117,
       1023120, 1023089], dtype=int64)

In [7]:
#bd[bd['comprobante'].isin(bdFiltro1["comprobante"])].sort_values(by="comprobante", ascending=True).tail(60)
bd=bdReducida[bdReducida["comprobante"].isin(bdFiltro1["comprobante"])].sort_values(by=["comprobante","dh"], ascending=True)
bd = bd.reset_index(drop=True)
#bd.head()

In [8]:
#list(bd.loc[55,["glosa"]])

In [9]:
#bd.to_excel("ux.xlsx")

In [10]:
bd=bd.drop_duplicates()
bd = bd.reset_index(drop=True)
#####bd

In [11]:
#bd["comprobante"].unique()

In [12]:
aux1=bd["comprobante"].unique()
#len(aux1)
#print(bd["comprobante"]==aux1[5])
#print(aux1[5])

consolidado=pd.DataFrame()

for q in range(len(aux1)):
    aux2 = bd[bd["comprobante"]==aux1[q]]
    aux2=aux2.reset_index(drop=True)
    data1=[]
    columnas=["Tipo","Comprobante","Concepto","Mayor","codMov","Movimiento","MN","Glosa","GlosaDetalle"]
    ###print("---", q, "-----")
    ###print(aux2.loc[:,["comprobante","mayor"]])
    ###print("-----")
    suma=0
    for i in aux2.index:
    
        if aux2["mayor"][i] in cuentas and (aux2["tipo"][i]=='V' or aux2["tipo"][i]=='D'):
            #print (i)
            suma=suma+aux2["mn"][i]/6.86
            
        else:
            #print ("else")
            for j in aux2.index:
                if i!=j:
                    if aux2["mn"][i]==aux2["mn"][j]:  ####### AQUI VER LA FORMA DE COMPARAR CON UN MARGEN

                        data1.append({
                            "Tipo":aux2["tipo"][i],"Comprobante":aux2["comprobante"][i],"Concepto":aux2["dh"][i],
                            "Mayor":aux2["mayor"][i],"codMov":aux2["codMov"][i],"Movimiento":aux2["nomMov"][i],
                            "MN":aux2["mn"][i],"Glosa":aux2["glosa"][i],"GlosaDetalle":aux2["glosaRenglon"][i],
                                    })
                        #print(q)
                        pares=pd.DataFrame(data1,columns=columnas)
                        #print(type(pares))      
    print ("la suma es ",suma, " c ", aux2["comprobante"][i])
        #suma=0

    ########################

    # reduce a una sola linea
    
    data2=[]
    columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
    def llenarData(concepto,a,b):
        if concepto !="Neteo":    
            data2.append({
                     "Tipo":pares["Tipo"][a],"Comprobante":pares["Comprobante"][a],"Concepto":concepto,
                    "Mayor1":pares["Mayor"][a],"Mayor2":pares["Mayor"][b],"codMov":pares["codMov"][b],
                    "Movimiento":pares["Movimiento"][b],"MontoUSD":round(pares["MN"][a]/6.86,2),
                    "Detalle":pares["GlosaDetalle"][b],"Glosa":pares["Glosa"][a]
                    })
        else:
            data2.append({
                    "Tipo":pares["Tipo"][a],"Comprobante":pares["Comprobante"][a],"Concepto":concepto,
                    "Mayor1":pares["Mayor"][a],"Mayor2":pares["Mayor"][b],"codMov":pares["codMov"][b],
                    "Movimiento":pares["Movimiento"][b],"MontoUSD":0, 
                    "Detalle":("±", round(pares["MN"][a]/6.86,2)),"Glosa":pares["Glosa"][a]
                    })
            
    #if len(pares)>2:
    #    print("Revisar oe")

    #for i in salida.index:
    ##print("que pasa")
    if pares["Mayor"][0] in cuentas and pares["Mayor"][1] in cuentas:
        #print("Posible Neteo")
        llenarData("Neteo",0,1)  ######## aqui ver para que el monto sea cero y que en el detalle salga el monto
        unRegistro=pd.DataFrame(data2,columns=columnas)
    else:     
        if pares["Mayor"][0] in cuentas and pares["Mayor"][1] not in cuentas and pares["Concepto"][0]=="D":
            llenarData("Ingreso",0,1)        
        if pares["Mayor"][0] in cuentas and pares["Mayor"][1] not in cuentas and pares["Concepto"][0]=="H":
            llenarData("Egreso",0,1)                   
        if pares["Mayor"][1] in cuentas and pares["Mayor"][0] not in cuentas and pares["Concepto"][1]=="D":
            llenarData("Ingreso",1,0)                    
        if pares["Mayor"][1] in cuentas and pares["Mayor"][0] not in cuentas and pares["Concepto"][1]=="H":            
            llenarData("Egreso",1,0) 

        unRegistro=pd.DataFrame(data2,columns=columnas)
      
    ## print(pares)
    ##print("-------------------")
    ##print("======= consolidado ============")
    ##print(consolidado) 
    #print("===================")
    consolidado=consolidado.append(unRegistro, ignore_index=True)  
    ##print ("la suma es ",suma, " c ", aux2["comprobante"][i])   
consolidado
#consolidado.to_excel("borrar.xlsx")


la suma es  0.011661807580174925  c  21762
la suma es  4453903.827988339  c  28350
la suma es  0.010204081632653062  c  28354
la suma es  0  c  1023079
la suma es  0  c  1023080
la suma es  0  c  1023081
la suma es  0  c  1023082
la suma es  0  c  1023083
la suma es  0  c  1023086
la suma es  0  c  1023089
la suma es  0  c  1023090
la suma es  0  c  1023099
la suma es  0  c  1023104
la suma es  0  c  1023106
la suma es  0  c  1023110
la suma es  0  c  1023117
la suma es  0  c  1023120
la suma es  0  c  1747823
la suma es  0  c  1747923
la suma es  0  c  1747924
la suma es  0  c  1747926
la suma es  0  c  1747929
la suma es  0  c  1747931
la suma es  0  c  1747933
la suma es  0  c  1747935
la suma es  0  c  1747937
la suma es  0  c  1747939
la suma es  0  c  1988999
la suma es  0  c  1989052
la suma es  0  c  1989068


,Tipo,Comprobante,Concepto,Mayor1,Mayor2,codMov,Movimiento,MontoUSD,Detalle,Glosa
0,S,1023080,Ingreso,11,606,1535,INTERESES OVERNIGHT CAPITAL TRABAJO ...,3583.34,NaN,'INTERESES DEVENGADOS SOBRE DEPOSITOS'||COMPLE...
1,S,1023080,Ingreso,11,606,1535,INTERESES OVERNIGHT CAPITAL TRABAJO ...,3583.34,NaN,'INTERESES DEVENGADOS SOBRE DEPOSITOS'||COMPLE...
2,S,1023082,Ingreso,17,606,6538,INTERESES SOBRE CUPONES PORTAFOLIO EURO ...,5981.49,NaN,||INTERESES DEVENGADOS DEL PORTAFOLIO GLOBAL E...
3,S,1023082,Ingreso,17,606,6538,INTERESES SOBRE CUPONES PORTAFOLIO EURO ...,5981.49,NaN,||INTERESES DEVENGADOS DEL PORTAFOLIO GLOBAL E...
4,S,1023086,Neteo,10,11,2902,INTERESES DEVENGADOS OVERNIGHT CAPITAL TRABAJO...,0.00,"(±, 150473.06)",'INTERESES GANADOS'||POR OVERNIGHT AUTOMATICOS...
5,S,1023089,Ingreso,10,404,7954,YPFB - EXPORTACION DE UREA Y OTROS ...,42930.00,POR CUENTA DE FERTIMAN INDUSTRIA E COMERCIO DE...,||TRANSFERENCIA DE FONDOS S/G. MENSAJES SWIFT ...
6,S,1023090,Ingreso,11,606,1535,INTERESES OVERNIGHT CAPITAL TRABAJO ...,3107.02,NaN,'INTERESES DEVENGADOS SOBRE DEPOSITOS'||OVERNI...
7,S,1023099,Ingreso,16,499,7355,VALORACION TITULOS DE LAS RESERVAS INTERNACION...,116.80,NaN,||REVALORACION DE TITULOS A PRECIOS DE MERCADO...
8,S,1023104,Egreso,10,405,3071,GARANTIA POR ACREDITIVOS - INST. DESC. ...,808036.80,(CC:H-0044417-2021) I-2021-10 VTA. DIVISAS S/G...,'RESPUESTA AL BANQUERO POR CANCELACION DE CART...
9,S,1023106,Ingreso,10,412,5832,03-H-307 DIREC.GRAL DE AERONAUTICA CIVIL-REC.P...,16238.50,POR CUENTA DE TAMPA CARGO SAS.,||TRANSFERENCIA DE FONDOS S/G. MENSAJES SWIFT ...


In [13]:
def llenarData(concepto,a,b):
    if concepto !="Neteo":    
        data2.append({
                 "Tipo":pares["Tipo"][a],"Comprobante":pares["Comprobante"][a],"Concepto":concepto,
                "Mayor1":pares["Mayor"][a],"Mayor2":pares["Mayor"][b],"codMov":pares["codMov"][b],
                "Movimiento":pares["Movimiento"][b],"MontoUSD":round(pares["MN"][a]/6.86,2),
                "Detalle":pares["GlosaDetalle"][b],"Glosa":pares["Glosa"][a]
                })
    else:
        data2.append({
                "Tipo":pares["Tipo"][a],"Comprobante":pares["Comprobante"][a],"Concepto":concepto,
                "Mayor1":pares["Mayor"][a],"Mayor2":pares["Mayor"][b],"codMov":pares["codMov"][b],
                "Movimiento":pares["Movimiento"][b],"MontoUSD":0, 
                "Detalle":("±", round(pares["MN"][a]/6.86,2)),"Glosa":pares["Glosa"][a]
                })

def dv(concepto,suma):
    data2.append({''
                 "Tipo":aux2["tipo"][0],"Comprobante":aux2["comprobante"][0],"Concepto":concepto,
                "Mayor1":"--","Mayor2":"--","codMov":"--",
                "Movimiento":"--","MontoUSD":round(suma,2),
                "Detalle":"--","Glosa":aux2["glosa"][0]
                })
    

In [14]:
aux1=bd["comprobante"].unique()
consolidado=pd.DataFrame()

for q in range(len(aux1)):
    aux2 = bd[bd["comprobante"]==aux1[q]]
    aux2=aux2.reset_index(drop=True)
    data1=[]
    columnas=["Tipo","Comprobante","Concepto","Mayor","codMov","Movimiento","MN","Glosa","GlosaDetalle"]
    #print("q ",q)
    #print(aux2["comprobante"][0])
    if aux2["tipo"][0]=='V' or aux2["tipo"][0]=='D': ####### los ajustes por arbitraje y revalorizacion ######
        data2=[]
        columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
        sumaDebe=0
        sumaHaber=0
        for i in aux2.index:
            if aux2["mayor"][i] in cuentas and aux2["dh"][i]=='D':
                sumaDebe=sumaDebe+aux2["mn"][i]/6.86
            elif aux2["mayor"][i] in cuentas and aux2["dh"][i]=='H':
                sumaHaber=sumaHaber+aux2["mn"][i]/6.86
        suma=sumaDebe-sumaHaber
        if suma>0:
            dv("Ingreso",suma)
            unRegistro=pd.DataFrame(data2,columns=columnas)
            consolidado=consolidado.append(unRegistro, ignore_index=True)
        else:
            dv("Egreso",-suma)
            unRegistro=pd.DataFrame(data2,columns=columnas)
            consolidado=consolidado.append(unRegistro, ignore_index=True)
    else: # este el caso normal digamos ##############################################################
        #print ("menos Tipo D o V")
        for i in aux2.index:    
            for j in aux2.index:
                if i!=j:
                    if aux2["mn"][i]==aux2["mn"][j]:
                        data1.append({
                            "Tipo":aux2["tipo"][i],"Comprobante":aux2["comprobante"][i],"Concepto":aux2["dh"][i],
                            "Mayor":aux2["mayor"][i],"codMov":aux2["codMov"][i],"Movimiento":aux2["nomMov"][i],
                            "MN":aux2["mn"][i],"Glosa":aux2["glosa"][i],"GlosaDetalle":aux2["glosaRenglon"][i],
                                    })
                        pares=pd.DataFrame(data1,columns=columnas)
                else:
                    if aux2['mayor'][i] in cuentas:
                        print(aux2['mayor'][i],'  ',aux2["comprobante"][i])
        data2=[]
        columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
        
        if pares["Mayor"][0] in cuentas and pares["Mayor"][1] in cuentas:
            #print("Posible Neteo")
            llenarData("Neteo",0,1)  ######## aqui ver para que el monto sea cero y que en el detalle salga el monto
            unRegistro=pd.DataFrame(data2,columns=columnas)
        else:     
            if pares["Mayor"][0] in cuentas and pares["Mayor"][1] not in cuentas and pares["Concepto"][0]=="D":
                llenarData("Ingreso",0,1)        
            if pares["Mayor"][0] in cuentas and pares["Mayor"][1] not in cuentas and pares["Concepto"][0]=="H":
                llenarData("Egreso",0,1)                   
            if pares["Mayor"][1] in cuentas and pares["Mayor"][0] not in cuentas and pares["Concepto"][1]=="D":
                llenarData("Ingreso",1,0)                    
            if pares["Mayor"][1] in cuentas and pares["Mayor"][0] not in cuentas and pares["Concepto"][1]=="H":            
                llenarData("Egreso",1,0) 

        unRegistro=pd.DataFrame(data2,columns=columnas)
        consolidado=consolidado.append(unRegistro, ignore_index=True)  

consolidado
#consolidado.to_excel("borrar.xlsx")

10    1023079
11    1023079
12    1023079
11    1023080
10    1023081
17    1023082
10    1023083
10    1023086
11    1023086
10    1023089
11    1023090
16    1023099
10    1023104
10    1023106
10    1023110
10    1023117
10    1023120
16    1747823
10    1747923
10    1747924
10    1747926
10    1747929
10    1747931
10    1747933
10    1747935
10    1747937
10    1747939
10    1747939
7    1988999
7    1989052
7    1989068


,Tipo,Comprobante,Concepto,Mayor1,Mayor2,codMov,Movimiento,MontoUSD,Detalle,Glosa
0,D,21762,Ingreso,--,--,--,--,0.00,--,AJUSTE COMPLEMENTARIO POR REVALORIZACION SALDO...
1,V,28350,Egreso,--,--,--,--,4453903.83,--,AJUSTE POR ARBITRAJE DE SALDOS DE ACTIVOS DE R...
2,V,28354,Egreso,--,--,--,--,0.01,--,AJUSTE COMPLEMENTARIO POR ARBITRAJE DE SALDOS ...
3,O,1989068,Ingreso,7,406,898,TGN CUENTA TRANSITORIA- M/E ...,50.00,NaN,"DEPOSITO DE EFECTIVO, DEPOSITANTE: REGINA MAMA..."
4,S,1023080,Ingreso,11,606,1535,INTERESES OVERNIGHT CAPITAL TRABAJO ...,3583.34,NaN,'INTERESES DEVENGADOS SOBRE DEPOSITOS'||COMPLE...
5,S,1023080,Ingreso,11,606,1535,INTERESES OVERNIGHT CAPITAL TRABAJO ...,3583.34,NaN,'INTERESES DEVENGADOS SOBRE DEPOSITOS'||COMPLE...
6,S,1023082,Ingreso,17,606,6538,INTERESES SOBRE CUPONES PORTAFOLIO EURO ...,5981.49,NaN,||INTERESES DEVENGADOS DEL PORTAFOLIO GLOBAL E...
7,S,1023082,Ingreso,17,606,6538,INTERESES SOBRE CUPONES PORTAFOLIO EURO ...,5981.49,NaN,||INTERESES DEVENGADOS DEL PORTAFOLIO GLOBAL E...
8,S,1023086,Neteo,10,11,2902,INTERESES DEVENGADOS OVERNIGHT CAPITAL TRABAJO...,0.00,"(±, 150473.06)",'INTERESES GANADOS'||POR OVERNIGHT AUTOMATICOS...
9,S,1023089,Ingreso,10,404,7954,YPFB - EXPORTACION DE UREA Y OTROS ...,42930.00,POR CUENTA DE FERTIMAN INDUSTRIA E COMERCIO DE...,||TRANSFERENCIA DE FONDOS S/G. MENSAJES SWIFT ...
